In [3]:
!pip install nltk

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import random
from operator import itemgetter

import pickle
import keras
from keras.preprocessing import sequence

import numpy as np

from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.externals import joblib

/opt/conda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
# Train all the models needed for generating comments and save them.
# These take awhile. So avoid running these if possible.
!python3 next_word_markov_model.py
!python3 predict_first_word_model.py
!python3 pedict_upvotes_model.py
!python3 cluster_comments.py
!python3 cluster_markov_model.py

In [5]:
# The generate_comment function generates a comment under the following premise.
# If a upvote model is provided, then we use the model to evaluate n comments
#  and chose the max. Otherwise we choose the first word generated
# If a first word model is provided, we use it to generate the seed. Otherwise,
#  the seed is randomly selected.
# If cluster model, a cluster markov model, and a tokenizer are provided
#  then we weight the next word probability distribution by distribution
#  generated by the cluster model. This attempts to push the model to
#  generate a on topic reply
# The generated comment is returned as a string
def generate_comment(parent_text, parent_sentiment_score,
                     words, generate_chain,
                     upvote_model=None, first_word_model=None,
                     comment_cluster_model=None, cluster_markov_model=None,
                     tokenizer=None, n=3, threshold=10):
    
    vader_analyzer = SentimentIntensityAnalyzer()
    if (comment_cluster_model is not None
        and cluster_markov_model is not None
        and tokenizer is not None):
            parent = " ".join([words[ind] for ind in parent_text if ind != 0])
            tokened_parent = tokenizer.texts_to_matrix(parent, mode="tfidf")
            cluster = comment_cluster_model.predict(tokened_parent)[0]
            parent_distribution = np.insert(cluster_markov_model[cluster], 0, 0)
    else:
        parent_distribution = None
    if first_word_model is None:
        choice = np.random.choice(chain[(0, 0)])
    else:
        choice = first_word_model_choice(parent_text,
                                         parent_sentiment_score,
                                         generate_chain,
                                         first_word_model, threshold)
    comment = np.array([0, 0, choice])
    while comment[-1] != -1 and len(comment) < 50: 
            #try:
            access_val = make_pdist(generate_chain[(comment[-2], comment[-1])], 10000)
            if upvote_model is not None:
                choice = upvote_model_choice(comment, access_val, words,
                                             generate_chain, vader_analyzer,
                                             upvote_model, n, parent_distribution)
            else:
                    choice = choose(access_val)                
            comment = np.append(comment, choice)
    return " ".join([words[ind] for ind in comment[2:-1]])
  

# Given a dictionary representing a probability distribution
# and the size of the probability distibution returns the
# a np array representation of the probability distribution
def make_pdist(pchain, vocab_size):
    temp = np.zeros(vocab_size + 1)
    for index, prob in pchain.items():
        temp[index + 1] = prob
    return temp

# If a parent distribution is provided, the current distribution
# is weighted by the parent distribution and then a choice is made
# from this new distribution. Otherwise a a choice is made from the
# current distribution
def choose(c_dist, p_dist=None):
    if p_dist is not None:
        p_dist[0] = c_dist[0] ** 2
        new_dist = p_dist * c_dist
        new_dist = new_dist / np.sum(new_dist)
    else:
        new_dist = c_dist
    return np.random.choice(np.arange(new_dist.shape[-1]), p=new_dist) - 1

# Uses the given local variables to generate threshold number of canidate words,
# and selects one that maximizes the upvotes predicted by the upvote predict model.
def upvote_model_choice(comment, access_val, word_index, chain,
                        vader_analyzer, upvote_model, threshold, parent_dist):
    choices = []
    chosen = []
    for i in range(0, threshold):
        choice = choose(access_val, parent_dist)            
        if choice in chosen:
            choice = choose(access_val, parent_dist)
        if choice == -1:
            comment_text = " ".join([word_index[ind] for ind in comment[2:]])
        else:
            comment_text = " ".join([word_index[ind] for ind in np.append(comment[2:], choice)])
        text_scores = vader_analyzer.polarity_scores(comment_text)
        text_scores_lst = [text_scores["neg"], text_scores["neu"],
                           text_scores["pos"], text_scores["compound"]]
        score = np.array([text_scores_lst])
        if choice == -1:
            padded_comment = sequence.pad_sequences([comment[2:]],maxlen=150)            
        else:
            padded_comment = sequence.pad_sequences([np.append(comment[2:],choice)],
                                                    maxlen=150)
        upvotes = upvote_model.predict([score, padded_comment])
        choices.append((choice, upvotes))
    return max(choices, key=itemgetter(1))[0]

# Attempts to generate a seed threshold times, and returns the seed if successfully generated,
# and returns the random seed otherwise
def first_word_model_choice(parent_text, parent_sentiment_score, chain, model, threshold):
    p_distribution = model.predict([np.array([parent_sentiment_score]) , np.array([parent_text])])[0]
    i = 0
    while i < threshold:
        choice = np.random.choice(np.arange(p_distribution.shape[-1]), p=p_distribution)
        if (0, choice) in chain:
            return choice
    return 0

In [6]:
# Load all models, data, words, and tokenizer
upvote_model = keras.models.load_model("predict_upvotes.h5")
first_word_model = keras.models.load_model("predict_first_word.h5")
cluster_comments_model = joblib.load(open("cluster_comments.pkl", 'rb'))
cluster_markov_model = pickle.load(open("cluster_markov_model.pkl", 'rb'))
p_chain = pickle.load(open("generator_probability_chain.pkl", 'rb'))

comment_texts = np.load("comment_texts.npy")
comment_sentiment_scores = np.load("comment_sentiment_scores.npy")
words = pickle.load(open("words.pkl", "rb"))
inv_map = {v: k for k, v in words.items()}
tokenizer = pickle.load(open("tokenizer.pkl", "rb"))

In [17]:
# Using all the previously described models and data to generate a random comment reply.
cmt = random.randint(0, comment_texts.shape[0])
print("comment = {}\n".format(
            " ".join([inv_map[ind] for ind in [word for word in comment_texts[cmt] if word != 0]])))

print("reply = {}".format(generate_comment(comment_texts[cmt], comment_sentiment_scores[cmt],
                                           inv_map, p_chain,
                                           upvote_model=upvote_model,
                                           first_word_model=first_word_model,
                                           comment_cluster_model=cluster_comments_model,
                                           cluster_markov_model=cluster_markov_model,
                                           tokenizer=tokenizer
                                           )
                         )
     )

comment = damn even here you're everywhere

reply = grep the term spyware like saying plagiarism is okay
